In [ ]:
import pymongo

For more details refer to 
 https://zenodo.org/record/5901956

Command used to export the data (this command takes about 15 minutes to complete).

`mongodump --db=JiraRepos --gzip --archive=mongodump-JiraRepos.archive`

Accompanying command to restore the data (this command takes about 15 minutes to complete). Expanded, this data is ~60GB inside MongoDB.

`mongorestore --gzip --archive=mongodump-JiraRepos.archive --nsFrom "JiraRepos.*" --nsTo "JiraRepos.*"`

Change the `--nsTo` command to contain the desired name for the JiraRepos database.
mongorestore --gzip --archive=mongodump-JiraRepos.archive --nsFrom "JiraRepos.*" --nsTo "JiraRepos.Apache"


For more information see: https://docs.mongodb.com/manual/tutorial/backup-and-restore-tools/

Jira Dataset for TD filtered was extracted from https://zenodo.org/record/5901956  (https://arxiv.org/pdf/2201.08368.pdf) and adapted 

Montgomery, Lloyd, Lüders, Clara, & Maalej, Prof. Dr. Walid. (2022). The Public Jira Dataset [Data set]. Zenodo. https://doi.org/10.5281/zenodo.5901956



In [ ]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")


In [ ]:
mydb = myclient["JiraRepos"]
mydb


In [ ]:
myapache = mydb["Apache"]
myjiraecosystem = mydb["JiraEcosystem"]
mySonatype = mydb["Sonatype"]
mymongo = mydb["MongoDB"]


In [ ]:
collist = mydb.list_collection_names()
collist


In [ ]:
import pandas as pd

In [ ]:
import pymongo as pm

CHUNK_SIZE = 500
client = pm.MongoClient()
coll = client.get_database('JiraRepos').get_collection('RedHat')
cursor = coll.find({}, batch_size=CHUNK_SIZE)


def yield_rows(cursor, chunk_size):
    """
    Generator to yield chunks from cursor
    :param cursor:
    :param chunk_size:
    :return:
    """
    chunk = []
    for i, row in enumerate(cursor):
        if i % chunk_size == 0 and i > 0:
            yield chunk
            del chunk[:]
        chunk.append(row)
    yield chunk


chunks = yield_rows(cursor, CHUNK_SIZE)
for chunk in chunks:
    df = pd.json_normalize(
        chunk, errors='ignore')

    df = df[["id", "fields.project.name", "fields.priority.name", "fields.created", "fields.labels", "fields.summary", "fields.description", "fields.status.name",
             "fields.status.description", "fields.issuetype.name", "fields.issuetype.description", "fields.issuetype.subtask", "fields.comments"]]

    df.to_csv(f"RedHat-{df.id[0]}.csv")

    

   

In [ ]:
import dask.dataframe as dd


In [ ]:
data_Sonatype = dd.read_csv("redhat/RedHat-*.csv")


In [ ]:
data_Sonatype = data_Sonatype.compute()


In [ ]:
data_Sonatype


In [ ]:
data_Sonatype1 = data_Sonatype.drop(columns="Unnamed: 0")


In [ ]:
data_Sonatype1


In [ ]:
data_Sonatype1["fields.issuetype.name"].value_counts()


In [ ]:
data_Sonatype1.reset_index(drop=True, inplace=True)


In [ ]:
data_Sonatype1

In [ ]:
data_Sonatype1.to_hdf('jira_all.h5', key="sonatype")


In [ ]:
df = pd.read_hdf('jira_all.h5', key="apache")
df1 = pd.read_hdf('jira_all.h5', key="mongodb")
df2 = pd.read_hdf('jira_all.h5', key="sonatype")
df3 = pd.read_hdf('jira_all.h5', key="jiraecosystem")
